In [11]:
# dependencies
from src.dataset import load, preprocess
from src import models
from src import utils

import tqdm

from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


BATCH_SIZE = 128
PCA_REDUCTION = 100
SELECTED_LANG = [24, 31, 41, 32, 12, 45]
NEPOCH = 20
P_VALIDATION=.7
cfg = {
    "num_layers":2,
    "weight":128
}

# Language Detection

Cette étude présente le dataset WiLI et tente de développer un modèle performant 

## Presentation du dataset

Le dataset est un extrait de différents paragraphes tirés de Wikipédia contenant 235 languages comprenant chaqun 1000 paragraphes. Le but est de détecter le language le plus présent dans un paragraphe.

Ce dataset est présenté dans "The WiLI benchmark dataset for written language identification" de Martin Thoma. Dans notre cas particulier, nous étudierons pas les langages artificiels, tel que le HTML ou XML, ni les langues mortes, tel que le Latin.

Dans la première partie, nous étudierons la classification de différents langages éloignés puis ensuite entre des langages sémantiquement rapprochés tel que l'Arabe et l'Arabe egyptien.

## Défis

Dans certains paragraphes, des caractères d'autres langages peuvent être présent. En citant un auteur étranger ou encore une référence à un nom commum etranger. Nous pouvons alors extraire les caratères les plus présents en les discrimiant par leur occurence comme indiqué par M. Thoma.

# 1. Load Data

In [12]:
# Load Data

train_ds, test_ds = load.execute(selected_lang=SELECTED_LANG)
print("train_ds: ", train_ds)
print("test_ds: ", test_ds)


Found cached dataset wili_2018 (C:/Users/etien/.cache/huggingface/datasets/wili_2018/WiLI-2018 dataset/1.1.0/78d7fe4a9d0a01168e45657f302c776ee0afc0978d44e2c3759f4c4975b845f5)
100%|██████████| 2/2 [00:00<00:00, 80.95it/s]
Loading cached processed dataset at C:\Users\etien\.cache\huggingface\datasets\wili_2018\WiLI-2018 dataset\1.1.0\78d7fe4a9d0a01168e45657f302c776ee0afc0978d44e2c3759f4c4975b845f5\cache-c149b0ff9e020866.arrow
Loading cached processed dataset at C:\Users\etien\.cache\huggingface\datasets\wili_2018\WiLI-2018 dataset\1.1.0\78d7fe4a9d0a01168e45657f302c776ee0afc0978d44e2c3759f4c4975b845f5\cache-99e8eb6a5f027ac5.arrow


train_ds:  Dataset({
    features: ['sentence', 'label'],
    num_rows: 3000
})
test_ds:  Dataset({
    features: ['sentence', 'label'],
    num_rows: 3000
})


## 1.1 Preprocessing

Tout d'abord, nous allons enlever les caractères spéciaux, transformer en caractère minuscule, réduire les set de caractères puis garder uniquement les mots avec les caractères les plus communs.

In [13]:
print("before removing: ", train_ds[0]['sentence'])

train_ds = preprocess.remove_special_char(train_ds)
train_ds = preprocess.lower_sentences(train_ds)

print("after removing: ", train_ds[0]['sentence'])




Loading cached processed dataset at C:\Users\etien\.cache\huggingface\datasets\wili_2018\WiLI-2018 dataset\1.1.0\78d7fe4a9d0a01168e45657f302c776ee0afc0978d44e2c3759f4c4975b845f5\cache-23c54f311f7dbf27.arrow
Loading cached processed dataset at C:\Users\etien\.cache\huggingface\datasets\wili_2018\WiLI-2018 dataset\1.1.0\78d7fe4a9d0a01168e45657f302c776ee0afc0978d44e2c3759f4c4975b845f5\cache-1b15aa0aab5b69ad.arrow


before removing:  Say San Jose et kumalima ya klase baley ed luyag na Northern Samar, Pilipinas. Unong ed 2010 Census, say papulasyon to et 16,079 totoo. Walay kabaleg tan sukat to ya 29.85 sq. km. Sikato et walad unaan ya distrito. Say zip code to et 6402.
after removing:  say san jose et kumalima ya klase baley ed luyag na northern samar pilipinas. unong ed  census say papulasyon to et  totoo. walay kabaleg tan sukat to ya . sq. km. sikato et walad unaan ya distrito. say zip code to et .


Pour appliquer ces transformations nous utilisons le traitement par batch du Dataset d'Huggingface.

## Vectorization

Pour vectorizer notre jeu de données, M. Thoma propose la mise en place d'un BagOfWord (BoW) pour compter les occurences des mots.
De plus, nous appliquons une analyse de composantes principales (PCA) pour réduire l'espace d'entrée.

In [14]:
train_ds, input_shape = preprocess.vectorize(train_ds, pca_reduction=PCA_REDUCTION)

explained_variance_ratio_:  0.5652721811929179


L'analyse en composante principale pour N=100 conserve 56,5% de la variance expliquée ce qui est suivant pour expliquer le jeu de données dans le cadre de la classification de langages.

Finally set format to be compatible with pytorch and define validation loader.

In [15]:
preprocess.set_format(train_ds)
length_train = int(P_VALIDATION*len(train_ds))
length_valid = int((1-P_VALIDATION)*len(train_ds))
while(length_train + length_valid) < len(train_ds):
    length_train +=1

train_subset, val_subset = torch.utils.data.random_split(
        train_ds, [length_train, length_valid], generator=torch.Generator().manual_seed(1))

train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE)
valid_loader = DataLoader(val_subset, batch_size=BATCH_SIZE)



# 2. Define model
Dans notre cas, nous allons implémenter un modèle récurrent LSTM et un modèle FeedForward en temps que benchmarck.

In [16]:
model = models.FeedForwardModel(cfg=cfg, input_size=input_shape, num_classes=len(SELECTED_LANG))
# model = models.LSTM(cfg=cfg, input_size=input_shape, num_classes=len(SELECTED_LANG))

### Loss

Etant donnés qu'il s'agit d'une classification multiclass, nous choisissons la CrossEntropyLoss en adaptant l'encodage des labels.

In [17]:
f_loss = nn.CrossEntropyLoss()

### Optimizer

Pour simplifier l'étude, nous fixons comme optimizer un optimizer Adam avec un learning rate de 0.001

In [18]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 3. Training

In [19]:
for e in (pbar := tqdm.tqdm(range(NEPOCH))):
        # Train 1 epoch
        train_loss = utils.train(model, train_loader, f_loss, optimizer, device)

        # Test
        valid_loss,accuracy = utils.test(model, valid_loader, f_loss, device)
        print("-----------------")
        pbar.set_description(f"valid_loss : {valid_loss:.2f}, accuracy: {accuracy:.2f}")

Train loss : 1.76: : 17it [00:00, 26.45it/s]
valid_loss : 1.68, accuracy: 0.29:   5%|▌         | 1/20 [00:00<00:17,  1.11it/s]

-----------------


Train loss : 1.57: : 17it [00:00, 30.73it/s]
valid_loss : 1.38, accuracy: 0.64:  10%|█         | 2/20 [00:01<00:14,  1.20it/s]

-----------------


Train loss : 1.30: : 17it [00:00, 26.61it/s]
valid_loss : 1.16, accuracy: 0.90:  15%|█▌        | 3/20 [00:02<00:14,  1.17it/s]

...

-----------------


Train loss : 1.07: : 17it [00:00, 26.61it/s]
valid_loss : 1.07, accuracy: 0.99:  95%|█████████▌| 19/20 [00:16<00:00,  1.17it/s]

-----------------


Train loss : 1.07: : 17it [00:00, 28.93it/s]
valid_loss : 1.07, accuracy: 0.99: 100%|██████████| 20/20 [00:16<00:00,  1.18it/s]

-----------------


# 4. Evaluation

In [20]:
test_ds = preprocess.remove_special_char(test_ds)
test_ds = preprocess.lower_sentences(test_ds)

test_ds, input_shape = preprocess.vectorize(test_ds, pca_reduction=PCA_REDUCTION, from_save=True)
preprocess.set_format(test_ds)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)
test_loss, accuracy = utils.test(model, test_loader, f_loss, device)
print(f"test_loss: {test_loss:.2f}, accuracy: {accuracy:.2f}")

Loading cached processed dataset at C:\Users\etien\.cache\huggingface\datasets\wili_2018\WiLI-2018 dataset\1.1.0\78d7fe4a9d0a01168e45657f302c776ee0afc0978d44e2c3759f4c4975b845f5\cache-a363a2486176307d.arrow
Loading cached processed dataset at C:\Users\etien\.cache\huggingface\datasets\wili_2018\WiLI-2018 dataset\1.1.0\78d7fe4a9d0a01168e45657f302c776ee0afc0978d44e2c3759f4c4975b845f5\cache-bdea94aca1829d8e.arrow


test_loss: 1.08, accuracy: 0.99


# 5. Conclusion

Les deux modèles étudiés performent de manière similaire sur un ensemble de langues de taille 6. Pour la suite, il serait pertinent de les comparer sur des langages très rapprochées et sur un ensemble plus élargie.

De plus pour comparer correctement les modèles, il aurait fallut utiliser WandB pour mieux visualiser les performances.

# 
